In [ ]:
# !pip install google-api-python-client
# !pip install textblob
# !pip install wordcloud matplotlib nltk

In [ ]:
from googleapiclient.discovery import build # For interacting with the YouTube API
import re # For extracting video ID from a URL
import pandas as pd
from textblob import TextBlob
import nltk
#nltk.download('stopwords')
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from nltk.corpus import stopwords

In [ ]:
# -------- Function: Extract Video ID from URL --------
def extract_video_id(url):
    """
    Extracts the 11-character YouTube video ID from a given URL.
    Supports both 'watch?v=' and 'youtu.be' formats.
    """
    pattern = r"(?:v=|\/)([0-9A-Za-z_-]{11})"
    match = re.search(pattern, url)
    return match.group(1) if match else None

In [ ]:
# -------- Function: Fetch Comments Using YouTube Data API --------
def get_comments(video_id, api_key, max_results=100):
    """
    Fetches up to 'max_results' top-level comments for the given video ID
    using the YouTube Data API v3.
    """
    youtube = build('youtube', 'v3', developerKey=api_key)
    comments = []
    next_page_token = None

    while len(comments) < max_results:
        response = youtube.commentThreads().list(
            part='snippet',
            videoId=video_id,
            maxResults=min(100, max_results - len(comments)),
            textFormat='plainText',
            pageToken=next_page_token
        ).execute()

        # Extract comment text
        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            comments.append(comment)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break

    return comments

In [ ]:
# -------- Main Script --------
# 🔑 Replace this with your actual YouTube Data API key
API_KEY = "API KEY HERE"

# Prompt the user to input a YouTube video link
url = input("🔗 Enter the YouTube video URL: ")

# Extract the video ID
video_id = extract_video_id(url)

if not video_id:
    print("⛔ Could not extract a valid video ID from the URL.")
else:
    print("📥 Fetching comments...")
    # Get up to 50 comments
    comments = get_comments(video_id, API_KEY, max_results=50)

    # Print the first 10 comments for confirmation
    for i, comment in enumerate(comments[:60], 1):
        print(f"{i}. {comment}")

In [ ]:
def analyze_sentiment(comments):
    """
    Analyze sentiment for each comment.
    Returns a list of (comment, polarity, sentiment) tuples.
    """
    results = []

    for comment in comments:
        blob = TextBlob(comment)
        polarity = blob.sentiment.polarity # from -1 (very negative) to +1 (very positive)

        # Label the sentiment
        if polarity > 0.1:
            sentiment = "positive"
        elif polarity < -0.1:
            sentiment = "negative"
        else:
            sentiment = "neutral"

        results.append((comment, polarity, sentiment))

    return results

In [ ]:
def summarize_sentiment(results):
    """
    Summarizes sentiment analysis results.
    Prints number and percentage of each sentiment,
    and gives a final overall sentiment conclusion.
    """
    # Convert to DataFrame for easy counting
    df = pd.DataFrame(results, columns=["Comment", "Polarity", "Sentiment"])
    total = len(df)

    positive = (df["Sentiment"] == "positive").sum()
    negative = (df["Sentiment"] == "negative").sum()
    neutral = (df["Sentiment"] == "neutral").sum()

    print("\n📊 Sentiment Summary:")
    print(f"Total Comments: {total}")
    print(f"✅ Positive: {positive} ({positive/total*100:.1f}%)")
    print(f"❌ Negative: {negative} ({negative/total*100:.1f}%)")
    print(f"⚪ Neutral : {neutral} ({neutral/total*100:.1f}%)")

    # Calculate average polarity
    avg_polarity = df["Polarity"].mean()
    print(f"\n🧮 Average Polarity Score: {avg_polarity:.2f}")

    # Final decision
    if avg_polarity > 0.1:
        print("🟢 Overall Sentiment: Mostly Positive")
    elif avg_polarity < -0.1:
        print("🔴 Overall Sentiment: Mostly Negative")
    else:
        print("🟡 Overall Sentiment: Mostly Neutral")

    return df

In [ ]:
results = analyze_sentiment(comments) # Step 1: analyze
df = summarize_sentiment(results) # Step 2: summarize

In [ ]:
def generate_wordcloud(comments, title="Word Cloud"):
    """
    Generate and display a word cloud from a list of comments,
    removing common stopwords.
    """
    # Combine all comments into one string
    text = " ".join(comments)

    # Use built-in English stopwords + NLTK stopwords
    custom_stopwords = set(STOPWORDS).union(set(stopwords.words('english')))

    # Generate the word cloud
    wordcloud = WordCloud(
        width=800,
        height=400,
        background_color='white',
        stopwords=custom_stopwords,
        max_words=200,
        collocations=False
    ).generate(text)

    # Display the image
    plt.figure(figsize=(12, 6))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(title, fontsize=18)
    plt.show()

In [ ]:
def split_comments_by_sentiment(results):
    """
    Splits comments into positive, negative, and neutral based on sentiment label.
    """
    positive_comments = [comment for comment, _, sentiment in results if sentiment == "positive"]
    negative_comments = [comment for comment, _, sentiment in results if sentiment == "negative"]
    neutral_comments = [comment for comment, _, sentiment in results if sentiment == "neutral"]
    return positive_comments, negative_comments, neutral_comments

In [ ]:
# 1. Split comments
positive_comments, negative_comments, neutral_comments = split_comments_by_sentiment(results)

# 2. Generate word clouds
generate_wordcloud(positive_comments, title=" Word Cloud - Positive Comments")
generate_wordcloud(negative_comments, title=" Word Cloud - Negative Comments")
generate_wordcloud(neutral_comments, title=" Word Cloud - Neutral Comments")